In [27]:
import pandas as pd
import plotly.express as px
from dstack import create_frame

AttributeError: module 'numpy' has no attribute 'ndarray'

In [11]:
url = "https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_19-covid-Confirmed.csv&filename=time_series_2019-ncov-Confirmed.csv"
df = pd.read_csv(url) # returns a "pandas dataframe"
df.head() # this function displays the first 5 rows of the dataframe; it helps check the format of the data and make sure that everything is correct

NameError: name 'pd' is not defined

In [12]:
cols = [df.columns[1]] + list(df.columns[-2:]) # this and below are a few of the ways how you can manipulate a dataframe using pandas
# country + two recent days (very simple, ignore week day, etc)
last_2_days = df[df["Province/State"].isnull()][cols].copy()
last_2_days # as you might've noticed above, the value of the expression in the end of the code cell is displayed in the output

NameError: name 'df' is not defined

In [13]:

d1 = last_2_days.columns[-1]
d2 = last_2_days.columns[-2]

last_2_days["delta"] = last_2_days[d1] - last_2_days[d2]
last_2_days["delta%"] = last_2_days["delta"] / last_2_days[d2]

NameError: name 'last_2_days' is not defined

In [14]:
last_2_days # displaying the resulting dataframe

NameError: name 'last_2_days' is not defined

In [15]:
min_cases = 50
# create frame and set stack name
top_speed_frame = create_frame("covid19/speed")
# top countries
sort_by_cols = ["delta", "delta%"]
for col in sort_by_cols:
    top = last_2_days[last_2_days[last_2_days.columns[1]]>min_cases].sort_values(by=[col], ascending=False).head(50)
    # commit attachment
    top_speed_frame.commit(top, f"Top 50 countries with the fastest growing number of confirmed Covid-19 cases (at least {min_cases})", {"Sort by": col})

top_speed_frame.push()

NameError: name 'create_frame' is not defined

In [16]:
cdf = df[(df["Country/Region"]=="Italy") & (df["Province/State"].isnull())][df.columns[4:]].T
cdf = cdf.rename(columns={cdf.columns[0]:"confirmed"}) # set the name of the new column resulted as a transposition of date dataframe columns

NameError: name 'df' is not defined

In [17]:
fig = px.line(cdf, x=cdf.index, y="confirmed")
fig.show() # displays the `plotly`'s figure

NameError: name 'px' is not defined

In [18]:
delta = (cdf.shift(-1) - cdf)
delta.tail() # display the last 5 rows of the dataframe to make sure the operation was correct

NameError: name 'cdf' is not defined

In [19]:
fig = px.line(delta, x=delta.index, y="confirmed")
fig.show()

NameError: name 'px' is not defined

In [20]:
def plots_by_country(country):
    cdf = df[(df["Country/Region"]==country) & (df["Province/State"].isnull())][df.columns[4:]].T
    cdf = cdf.rename(columns={cdf.columns[0]:"confirmed"})
    cfig = px.line(cdf, x=cdf.index, y="confirmed")
    delta = (cdf.shift(-1) - cdf).rename(columns={"confirmed": "confirmed per day"})
    cdfig = px.line(delta, x=cdf.index, y="confirmed per day")
    delta_p = ((cdf.shift(-1) - cdf) / cdf.shift(-1)).rename(columns={"confirmed": "confirmed per day %"})
    cdpfig = px.line(delta_p, x=cdf.index, y="confirmed per day %")
    return (cfig, cdfig, cdpfig)

In [21]:
(fig1, fig2, fig3) = plots_by_country("Austria")
fig1.show()
fig2.show()
fig3.show()

NameError: name 'df' is not defined

In [22]:
# get top 30 countries by the number of new cases on the last day
countries = df[df["Province/State"].isnull()].sort_values(by=[df.columns[-1]], ascending=False)[["Country/Region"]].head(30)

# create a frame and iterate over the top countries to commit three plots for every country: new absolute cases, increase in absolute numbers, and increase in percent
frame = create_frame("covid19/speed_by_country")
for c in countries["Country/Region"].tolist():
    print(c)
    (fig1, fig2, fig3) = plots_by_country(c)
    frame.commit(fig1, f"Confirmed cases in {c}", {"Country": c, "Chart": "All cases"})
    frame.commit(fig2, f"New confirmed cases in {c}", {"Country": c, "Chart": "New cases"})
    frame.commit(fig3, f"New confirmed cases in {c} in %", {"Country": c, "Chart": "New cases (%)"})

frame.push()

NameError: name 'df' is not defined

In [23]:
# filter Italy, transpose date dataframe columns into dataframe rows
t1 = df[(df["Country/Region"]=="Italy") & (df["Province/State"].isnull())][df.columns[4:]].T
# set the new column name
t1 = t1.rename(columns={t1.columns[0]:"confirmed"})
# make the dataframe's index a regular column; we'll later need it to highlight each country with its own color
t1.reset_index()
# add country column
t1["Country/Region"] = "Italy"
t1.tail() # display the last 5 rows to make sure everything is correct

NameError: name 'df' is not defined

In [24]:

# this function return three dataframes: absolute new cases, absolute increase, percent increase
def country_df(country):
    cdf = df[(df["Country/Region"]==country) & (df["Province/State"].isnull())][df.columns[4:]].T
    cdf = cdf.rename(columns={cdf.columns[0]:"confirmed"})
    delta = (cdf.shift(-1) - cdf).rename(columns={"confirmed": "confirmed per day"})
    delta.reset_index()
    delta["Country/Region"] = country
    delta_p = ((cdf.shift(-1) - cdf) / cdf.shift(-1)).rename(columns={"confirmed": "confirmed per day %"})
    delta_p.reset_index()
    delta_p["Country/Region"] = country
    cdf.reset_index()
    cdf["Country/Region"] = country
    return (cdf, delta, delta_p)

In [25]:
# top 10 countries by last day absolute
top10 = df[df["Province/State"].isnull()].sort_values(by=[df.columns[-1]], ascending=False)[["Country/Region"]].head(10)

# make a single lists of dataframes for all countries
top = []
top_delta = []
top_delta_p = []
for c in top10["Country/Region"].tolist():
    (x, y, z) = country_df(c)
    top.append(x)
    top_delta.append(y)
    top_delta_p.append(z)

test = pd.concat(top) # make a pandas dataframe out for the new cases
# plot the resulted dataframe of new cases to make sure everything is correct
px.line(test, x=test.index, y="confirmed", color='Country/Region').show()

NameError: name 'df' is not defined

In [26]:
frame = create_frame("covid19/speed_by_country_all")

top10df = pd.concat(top)
fig = px.line(top10df, x=top10df.index, y="confirmed", color='Country/Region')
frame.commit(fig, "Confirmed cases in top 10 countries", {"Country": "Top 10", "Chart": "All cases"})

top10df_delta = pd.concat(top_delta)
fig = px.line(top10df_delta, x=top10df_delta.index, y="confirmed per day", color='Country/Region')
frame.commit(fig, "New confirmed cases in top 10 countries", {"Country": "Top 10", "Chart": "New cases"})

top10df_delta_p = pd.concat(top_delta_p)
fig = px.line(top10df_delta_p, x=top10df_delta_p.index, y="confirmed per day %", color='Country/Region')
frame.commit(fig, "New confirmed cases in top 10 countries in %", {"Country": "Top 10", "Chart": "New cases (%)"})

for c in countries["Country/Region"].tolist():
    print(c)
    (fig1, fig2, fig3) = plots_by_country(c)
    frame.commit(fig1, f"Confirmed cases in {c}", {"Country": c, "Chart": "All cases"})
    frame.commit(fig2, f"New confirmed cases in {c}", {"Country": c, "Chart": "New cases"})
    frame.commit(fig3, f"New confirmed cases in {c} in %", {"Country": c, "Chart": "New cases (%)"})

frame.push()

NameError: name 'create_frame' is not defined